# agent built with langchain&qwen

In [ ]:
import os
import json
from langchain_community.tools.tavily_search import TavilySearchResults
from modelscope import AutoTokenizer,AutoModelForCausalLM

os.environ['TAVILY_API_KEY']='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'    # travily搜索引擎api key

llm

In [ ]:
model_name='qwen/Qwen-14B-Chat-Int4'
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
llm=AutoModelForCausalLM.from_pretrained(model_name,device_map="auto",trust_remote_code=True).eval()

tools

In [ ]:
tools=[
    TavilySearchResults(max_results=5), # travily搜索引擎
]  
tool_names='or'.join([tool.name for tool in tools])  # 拼接工具名
tool_descs=[] # 拼接工具详情
for t in tools:
    args_desc=[]
    for name,info in t.args.items():
        args_desc.append({'name':name,'description':info['description'],'type':info['type']})
    args_desc=json.dumps(args_desc,ensure_ascii=False)
    tool_descs.append('%s: %s,args: %s'%(t.name,t.description,args_desc))
tool_descs='\n'.join(tool_descs)

In [ ]:
print(tool_names)
print(tool_descs)

prompt

In [ ]:
chat_history=[]
prompt_tpl='''Answer the following questions as best you can. You have access to the following tools:

{tool_descs}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can be repeated zero or more times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {query}
{agent_scratchpad}
'''

In [ ]:
# 测试prompt
prompt=prompt_tpl.format(tool_descs=tool_descs,tool_names=tool_names,query='查一下明天青岛的天气',agent_scratchpad='')
print(prompt)

agent execution

In [ ]:
def agent_execute(query,chat_history=[]):
    global tools,tool_names,tool_descs,prompt_tpl,llm,tokenizer
    
    agent_scratchpad='' # agent执行过程
    while True:
        # 保留最近的历史对话，控制显存
        chat_history=chat_history[-3:]
        # 1）触发llm思考下一步action
        prompt=prompt_tpl.format(tool_descs=tool_descs,tool_names=tool_names,query=query,agent_scratchpad=agent_scratchpad)
        print('prompt: %s\n\033[32magent正在思考... ...'%prompt)
        response,chat_history=llm.chat(tokenizer,prompt,history=chat_history,stop_words_ids=[tokenizer.encode('Observation:')],top_k=1) # 模型稳定返回，即只选择概率最高的token
        print('---LLM raw response\n%s\n\033[0m---'%response)
        # 2）解析thought+action+action input+observation or thought+final answer
        thought_i=response.rfind('Thought:')
        final_answer_i=response.rfind('\nFinal Answer:')
        action_i=response.rfind('\nAction:')
        action_input_i=response.rfind('\nAction Input:')
        observation_i=response.rfind('\nObservation:')
        # 3）返回final answer，执行完成
        if final_answer_i!=-1 and thought_i<final_answer_i:
            print('\033[34mfinal answer: \033[0m%s\n\033[34m\n'%(response[final_answer_i+len('\nFinal Answer:'):]))
            return True,response[final_answer_i+len('\nFinal Answer:'):],chat_history
        # 4）解析action
        if not (thought_i<action_i<action_input_i<observation_i):
            print('\033[31m LLM回复格式异常, 放弃继续思考.\nresponse:%s\n \033[0m'%response)
            return False,'LLM回复格式异常',chat_history
        thought=response[thought_i+len('Thought:'):action_i].strip()
        action=response[action_i+len('\nAction:'):action_input_i].strip()
        action_input=response[action_input_i+len('\nAction Input:'):observation_i].strip()
        print('\033[34mthought: \033[0m%s\n\033[34maction: \033[0m%s\n\033[34maction_input: \033[0m%s\n'%(thought,action,action_input))
        # 5）匹配tool
        the_tool=None
        for t in tools:
            if t.name==action:
                the_tool=t
                break
        if the_tool is None:
            observation='the tool not exist'
            agent_scratchpad=agent_scratchpad+response+observation+'\n'
            continue 
        # 6）执行tool
        try:
            action_input=json.loads(action_input)
            tool_ret=the_tool.invoke(input=action_input)
        except Exception as e:
            observation='the tool has error:{}, we can try once more'.format(e)
        else:
            observation=str(tool_ret)
        agent_scratchpad=agent_scratchpad+response+observation+'\n'

test

In [ ]:
my_history=[]
while True:
    query=input('query:')
    success,result,my_history=agent_execute(query,chat_history=my_history)
    my_history=my_history[-3:]